In [1]:
# !pip install transformers
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# !pip install sentence-transformers
# !pip install datasets

In [2]:
import numpy as np
import pandas as pd
import ast
import warnings
import scipy
import matplotlib.pyplot as plt
import spacy
import difflib
import tqdm
import json
import pickle
import logging
import itertools
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from datasets import Dataset
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

/Users/gke/py312env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-20 17:57:58,941 : INFO : PyTorch version 2.3.1 available.


In [50]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
tokenizer = model.tokenizer

dfn = pd.read_csv('lmsys-chatbot-arena/aux_files/dfn.csv')

2024-06-20 18:19:49,152 : INFO : Use pytorch device_name: mps
2024-06-20 18:19:49,153 : INFO : Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
/Users/gke/py312env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
dfn.head(1)

,id,prompt,model_a_answer,model_b_answer,winner
0,30192,Is it morally right to try to have a certain p...,The question of whether it is morally right to...,"As an AI, I don't have personal beliefs or opi...",A


In [58]:
ids = []
anchor = []
positive = []
idx = 0
def create_id_anchor_positive(id_ , anch , posit):
    ids.append(id_)
    anchor.append(anch)
    positive.append(posit)

for row in dfn.iterrows():
    prompt = row[1]['prompt']
    answer_a = row[1]['model_a_answer']
    answer_b = row[1]['model_b_answer']
    
    p = tokenizer([prompt] , return_tensors='pt').encodings[0].tokens
    a = tokenizer([answer_a] , return_tensors='pt').encodings[0].tokens
    b = tokenizer([answer_b] , return_tensors='pt').encodings[0].tokens
    
    p_len = len(p)
    a_len = len(a)
    b_len = len(b)

    id_ = row[1]['id']
    winner = row[1]['winner']

    if p_len > 256:
        p_prime = [p[i:i+254] for i in range(1, len(p)-1, 254)]
        s_temp = []
        for s in p_prime:
            s = tokenizer.convert_tokens_to_string(s)
            s_temp.append(s)
        
        if winner == 'N':
            a.pop()
            a_prime = [a[i:i+254] for i in range(1, len(a)-1, 254)]
            a_temp = []
            for s in a_prime:
                s = tokenizer.convert_tokens_to_string(s)
                a_temp.append(s)
        if winner == 'B':
            b_prime = [b[i:i+254] for i in range(1, len(b)-1, 254)]
                # create_id_anchor_positive(id_ , winner , answer_a)
        m = itertools.product(s_temp , a_temp)
        for zed in m:
            print(zed)
            break
        break

    elif a_len > 256:
        pass
    elif b_len > 256:
        pass
    else:
        if winner == 'A':
            create_id_anchor_positive(id_ , prompt , answer_a)
        if winner == 'B':
            create_id_anchor_positive(id_ , prompt , answer_b)
        pass
    idx += 1

('given a passage and some supplementary information, you are required to correct and output the refined passage in a fluent and natural style, following these rules : 1. the supplementary information may include the following parts : " counting " information that specifies how many instances of a certain kind of entity exist ; 2. correct the sentences in the passage if they are inconsistent with the supplementary information. 3. note that instances of a certain category can also belong to its super - catergories. for example, a bus is also a car. if there is no head in the picture, you cannot see the mask and glasses. examples : supplementary information : counting : there are 2 car. passage : there is no car in the image. the image features a red double - decker bus. refined passage : yes, there are cars in the image. the image features a red double - decker bus. - - - - - - - - - - - - - - - - - - - supplementary information : counting : there are no shoes. passage : the man is wear